In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.agents.agent_toolkits.file_management import FileManagementToolkit
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from dotenv import load_dotenv

import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

llm = OpenAI(temperature=0, max_tokens=512)

file_managmenet_toolkit = FileManagementToolkit(
    root_dir="./code/",
    selected_tools=["file_search", "read_file", "write_file"]
)

tools = file_managmenet_toolkit.get_tools()

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True)

In [ ]:
template = """
You are an expert react developer.
Create a react {type} page that has the folloing required files: {fields}
"""
prompt_template = PromptTemplate(
  template=template,
  input_variables=["type", "fields"]
)

create_page = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")
content = create_page.run({"type": "login", "fields": "first name, last name, email"})
print(content)

In [ ]:
template = """
find the {type} file and write the following content to it:
{content}
"""
prompt_template = PromptTemplate(
  template=template,
  input_variables=["type", "content"]
)

fmt_prompt = prompt_template.format(type='login', content=content)
print(fmt_prompt)

In [ ]:
agent.run(fmt_prompt)